In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Library and Loading Datsets

In [ ]:
import os
import numpy as np
import pandas as pd


#Plot

import seaborn as sns
import matplotlib.pyplot as plt

#Encoding
from sklearn.preprocessing import LabelEncoder 

#Modelling and Cross-Validation
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

#Ensemble
from scipy import stats



In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv', index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv', index_col='row_id')
sub = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Explore Data

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
print('Columns:\n{}'.format(list(train.columns)))

# Basic data Check

In [ ]:
print('Train data shape:', train.shape)
print('Test data shape:', test.shape)

# Missing Values

In [ ]:
train.isnull().any().sum()

In [ ]:
test.isnull().any().sum()

# Duplicates

In [ ]:
train.duplicated().sum()

In [ ]:
test.duplicated().sum()

# Sample Weights

In [ ]:
sample_weight = train.value_counts().values

In [ ]:
len(sample_weight)

# Dropping Duplicated Rows

In [ ]:
train_with_duplicates = train.copy()
train.drop_duplicates(keep='first', inplace = True)
duplicates_train = train.duplicated().sum()
print(train.shape)
print(train.duplicated().sum())

# Features

Numeric Features

In [ ]:
numerical_features = train.drop(columns = 'target')


In [ ]:
print(list(numerical_features))

# Target Distribution

In [ ]:
def plot(train, title):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.xticks(rotation=30, ha='right')
    ax = sns.countplot(x=train['target'], data=train)
    print(pd.Series(train['target'], index=train.index).value_counts().sort_index()*100/len(train['target']))

In [ ]:
plot(train, "Target distribution without duplicates")

In [ ]:
plot(train_with_duplicates, "Target distribution with duplicates")

# Reduce Memory Usage

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
reduce_mem_usage(train_with_duplicates)
reduce_mem_usage(train)
reduce_mem_usage(test)
reduce_mem_usage(sub)

# Modelling

In [ ]:
folds = StratifiedKFold(n_splits=10 , random_state=1, shuffle=True)

In [ ]:
n_splits = 10
estimators = 300

target_encoder = LabelEncoder()
def run_model(train, sample_weight = []):
    df = train.copy()
    df['target'] = target_encoder.fit_transform(df['target'])
    
    X=df.drop('target', axis=1)
    y=df.target
    
    scores = []
    y_probs = []
    
    folds = StratifiedKFold(n_splits=n_splits , random_state=1, shuffle=True)
    for fold, (train_id, test_id) in enumerate(folds.split(X,y)):
        
        X_train = X.iloc[train_id]
        y_train = y.iloc[train_id]
        X_valid = X.iloc[test_id]
        y_valid = y.iloc[test_id]
        
        model = ExtraTreesClassifier(
            n_estimators = estimators,
            random_state = 1,
            n_jobs = -1
        )
        
        if len(sample_weight) == 0:
            model.fit(X_train,y_train)
        else:
            sample_weight_train = sample_weight[train_id]
            model.fit(X_train,y_train, sample_weight_train)
            
        valid_pred = model.predict(X_valid)
        
        valid_score = 0
        if len(sample_weight) == 0:
            valid_score = accuracy_score(y_valid, valid_pred)
        
        else:
            sample_weight_valid = sample_weight[test_id]
            valid_score =  accuracy_score(y_valid, valid_pred, sample_weight= sample_weight_valid)
            
            
        print("Fold:", fold+1, "Accuracy", valid_score)
        scores.append(valid_score)
        y_probs.append(model.predict_proba(test))
        
    print("Mean accuracy score:", np.array(scores).mean())
    return y_probs

# Without duplicates(using sample weight) 

In [ ]:
y_probs = run_model(train, sample_weight)

# Without duplicates and sample weight

In [ ]:
y_probs_without_sample_weight = run_model(train)

# Without duplicates and sample weight 

In [ ]:
y_prob_with_duplicates = run_model(train_with_duplicates)

# Postprocessing 

In [ ]:
def post_processing(y_probs, train, tune =[]):
    
    y_prob = sum(y_probs) / len(y_probs)
    target_distribution= train['target'].value_counts().sort_index() / len(train)*100
    def get_diff(tune):
        y_pred_tuned = target_encoder.inverse_transform(np.argmax(y_prob + tune, axis=1))
        return target_distribution - pd.Series(y_pred_tuned).value_counts().sort_index() / len(test)* 100
    if len(tune) == 0:
        tune = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        diff = get_diff(tune)
        while abs(diff).max() > 0.1:
            for i in range(len(diff)):
                if diff[i] > 0.1:
                    tune[i] += .001
                    break
                if diff[i] < -0.1:
                    tune[i] -= .001
                    break
            diff = get_diff(tune)
    print(tune)
    y_pred_tuned = target_encoder.inverse_transform(np.argmax(y_prob + tune, axis=1))
    print(pd.Series(y_pred_tuned, index=test.index).value_counts().sort_index() / len(test)*100)
        
    return y_pred_tuned
    
    

Apply the post processing step using a predetermined tune value or by trying to find a tune value. So, it will be possible to see difference between those two approaches.

Also, it will be possible comparing three models run with or without duplicates (also with sample weight) when the tune value is the same.

In [ ]:
tune = [0, 0, 0.01, 0.03, 0, 0, 0, 0, 0, 0]

# Without duplicates (using sample weight)

In [ ]:
y_pred_tuned = post_processing(y_probs, train)
y_pred_tuned_with_tune = post_processing(y_probs, train, tune)

# Without duplicates and sample weight

In [ ]:
y_pred_tuned_without_sample_weight = post_processing(y_probs_without_sample_weight, train)
y_pred_tuned_without_sample_weight_with_tune = post_processing(y_probs_without_sample_weight, train, tune)

# With duplicates

In [ ]:
y_pred_tuned_with_duplicates = post_processing(y_prob_with_duplicates, train_with_duplicates)
y_pred_tuned_with_duplicates_with_tune = post_processing(y_prob_with_duplicates, train_with_duplicates, tune)

# Intersection between Training and Test sets

It appears there are some rows which are same both in training and test set.
So, at least be sure that we are not predicting them wrong.

In [ ]:
# According to XYZT, there are 1521 rows which are both included in the
# training and test data. 486 of them are unique and contained by the
# test data. So, find those rows, keep their indices and use them to
# update our submissions if there is any difference.
intersection = test.copy()
intersection['copy_index'] = intersection.index
intersection = intersection.merge(train, on=list(test.columns), how="inner")
print("There are {0} rows which are included in both training and test sets.".format(len(intersection)))

In [ ]:
def update_target(data, intersection):
    count = 0
    for index, row in intersection.iterrows():
        if data[int(row['copy_index']) - 200000] != row.target:
            count = count + 1
            data[int(row['copy_index']) - 200000] = row.target
    print("Updated {0} rows.".format(count))

In [ ]:
update_target(y_pred_tuned, intersection)
update_target(y_pred_tuned_with_tune, intersection)
update_target(y_pred_tuned_without_sample_weight, intersection)
update_target(y_pred_tuned_without_sample_weight_with_tune, intersection)
update_target(y_pred_tuned_with_duplicates, intersection)
update_target(y_pred_tuned_with_duplicates_with_tune, intersection)

# Submission

In [ ]:
def submission(name, y_pred_tuned):
    sub["target"] = y_pred_tuned
    sub.to_csv(name, index=False)

# Without duplicates (using sample weight)
submission("submission.csv", y_pred_tuned)
submission("submission_with_tune.csv", y_pred_tuned_with_tune)

# Without duplicates and sample weight
submission("submission_without_sample_weight.csv", y_pred_tuned_without_sample_weight)
submission("submission_without_sample_weight_with_tune.csv", y_pred_tuned_without_sample_weight_with_tune)

## With duplicates
submission("submission_with_duplicates.csv", y_pred_tuned_with_duplicates)
submission("submission_with_duplicates_with_tune.csv", y_pred_tuned_with_duplicates_with_tune)

# Ensemble

In [ ]:
preds = []
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        if (dirname != '/kaggle/input/tabular-playground-series-feb-2022') & ('.csv' in filename):
            df = pd.read_csv(os.path.join(dirname, filename))
            preds.append(df['target'])
submission("ensemble.csv", stats.mode(np.array(preds), axis=0)[0].transpose())